# Import Dependencies

In [1]:
import pandas as pd
import numpy as np

# Parse First Text File

In [2]:
file_path = 'llama_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df1 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df1)/total_questions)*100)
result1

'Percentage of Questions with Executable Code: 56.048166392993984'

In [3]:
#this needs to be rewritten
def check_topic(df):
    count = 0
    for i in range(0, len(df)):
        if df.iloc[i]['topic'] in df.iloc[i]['question']:
            count+=1
            
    return "Percentage of Executable Code that Contains Topic: " + str((count/len(df))*100)
        
    
result2 = check_topic(df1)
result2

'Percentage of Executable Code that Contains Topic: 83.7890625'

In [4]:
print(f"Percentage of questions that have 0 as answer: {(len(df1[df1['answer']==0])/len(df1))*100}")

Percentage of questions that have 0 as answer: 6.15234375


In [5]:
df1 = df1.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df1)):
    question_numbers.append(f"llama_q{i}")
df1['question_number'] = question_numbers
#df1_an = df1.sample(n=250, random_state = 42)

In [6]:
df1

,question,solution,answer,topic,question_number
0,Barbie has 500 pairs of shoes. 200 are high he...,def solution():\n #Barbie has 500 pairs of ...,150,Barbie,llama_q0
1,Black Panther can run 200 meters in 20 seconds...,def solution():\n #Black Panther can run 20...,0.2,Black Panther,llama_q1
2,A cookie baker has 2500 cookies. \nHe sells 10...,def solution():\n #The baker started with 2...,800,cookies,llama_q2
3,Iron Man has a total of 42 suits. 25 of the su...,def solution():\n #Iron Man has a total of ...,0,Iron Man,llama_q3
4,A pizza has 8 slices. 2 of the slices have mus...,def solution():\n #A pizza has 8 slices\n ...,2,pizza,llama_q4
...,...,...,...,...,...
1019,A cookie factory produces 1000 cookies per day...,def solution():\n #A cookie factory produce...,416.666667,cookies,llama_q1013
1020,Naruto has 1000 ryo. He spends 100 ryo on a ne...,def solution():\n #Naruto has 1000 ryo\n ...,400,Naruto,llama_q1014
1021,"Thor, the God of Thunder, has 3000 thunderbolt...",def solution():\n #Thor started with 3000 t...,2000,"Thor, the God of Thunder",llama_q1015
1022,Captain Marvel can fly at 120 miles per hour. ...,def solution():\n #Captain Marvel can fly a...,1.0,Captain Marvel,llama_q1016


# Parse Second Text File

In [7]:
file_path = 'mathwell_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df2 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df2)/total_questions)*100)
result1

'Percentage of Questions with Executable Code: 65.36550745209368'

In [8]:
result2 = check_topic(df2)
result2

'Percentage of Executable Code that Contains Topic: 79.80456026058633'

In [9]:
print(f"Percentage of questions that have 0 as answer: {(len(df2[df2['answer']==0])/len(df2))*100}")

Percentage of questions that have 0 as answer: 6.948968512486428


In [10]:
df2 = df2.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df2)):
    question_numbers.append(f"mathwell_q{i}")
df2['question_number'] = question_numbers
#df2_an = df2.sample(n=250, random_state = 42)

In [11]:
df2

,question,solution,answer,topic,question_number
0,The Cincinnati Bengals and the Kansas City Chi...,def solution():\n #The Bengals have 42 poin...,97,football,mathwell_q0
1,Serena Williams has 24 Grand Slam singles titl...,def solution():\n #Serena Williams has 24 G...,2,Serena Williams,mathwell_q1
2,1000 unicorns are in the forest. 200 of them a...,def solution():\n #1000 unicorns are in the...,500,unicorns,mathwell_q2
3,A T-Rex is 20 feet tall. A Brontosaurus is 10...,def solution():\n #A T-Rex is 20 feet tall\...,15.0,dinosaurs,mathwell_q3
4,The lacrosse team has 120 sticks. 20 of the st...,def solution():\n #Total sticks\n sticks...,0,lacrosse,mathwell_q4
...,...,...,...,...,...
916,A pizza is cut into 12 slices. 1/3 of the slic...,def solution():\n #A pizza is cut into 12 s...,2.0,pizza,mathwell_q886
917,Naruto ate 12 ramen noodles for lunch. He ate ...,def solution():\n #Naruto ate 12 ramen nood...,15,Naruto,mathwell_q887
918,Batman has 5000 Batarangs. He uses 1000 of the...,def solution():\n #Batman has 5000 Batarang...,1000,Batman,mathwell_q888
919,Iron Man has 1000 nuts and bolts. He uses 200 ...,def solution():\n #Iron Man started with 10...,700,Iron Man,mathwell_q889


# Parse Third Text File

In [12]:
file_path = 'llema_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    if 'while' in solution:
                        continue
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                if 'while' in solution:
                    continue
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df3 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df3)/total_questions)*100)
result1

8
Captain America was 1.0 times higher than he was wide.
60
There are at least one cookie left in the cookie jar


'Percentage of Questions with Executable Code: 23.695388349514563'

In [13]:
result2 = check_topic(df3)
result2

'Percentage of Executable Code that Contains Topic: 70.16645326504481'

In [14]:
print(f"Percentage of questions that have 0 as answer: {(len(df3[df3['answer']==0])/len(df3))*100}")

Percentage of questions that have 0 as answer: 4.481434058898848


In [15]:
df3 = df3.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df3)):
    question_numbers.append(f"llema_q{i}")
df3['question_number'] = question_numbers
#df3_an = df3.sample(n=250, random_state = 42)

In [16]:
df3

,question,solution,answer,topic,question_number
0,Fred has 4 dogs. 3 of Fred's dogs have 4 legs ...,def solution():\n #Fred had 4 dogs\n dog...,15,dogs,llema_q0
1,At a high school football game there are 5000 ...,def solution():\n #Total number of fans\n ...,2500.0,cheer leading,llema_q1
2,A litter of kittens has 5 kittens. They want t...,def solution():\n #A litter of kittens has ...,10,cats,llema_q2
3,"The Buffalo Bandits have 10 attackers, 15 defe...",def solution():\n #The Buffalo Bandits have...,52,lacrosse,llema_q3
4,"Daddy wants to eat pizza, but not too much. He...",def solution():\n #A slice is 1 piece of pi...,3285,pizza,llema_q4
...,...,...,...,...,...
776,Superman can punch 15 criminals per minute. If...,def solution():\n #Superman can punch 15 cr...,195,Superman,llema_q773
777,A soccer team has 22 players. 10 players are f...,def solution():\n #There are 22 players on ...,12,soccer,llema_q774
778,Captain America can catch 30 criminals in 30 s...,def solution():\n #Captain America can catc...,30.0,Captain America,llema_q775
779,Creeper and Cleo are cats. Creeper has 1000 ha...,def solution():\n #Creeper has 1000 hairs\n...,3550,cats,llema_q776


# Parse Fourth Text File

In [17]:
file_path = 'mammoth_questions.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df4 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df4)/total_questions)*100)
result1

92
250
16
2
730
22
9
960
24
20
200
150
9
13333.333333333334
325.0
96
24
8
9
86400
16.666666666666668
3
86
10
50.0
5
1
3276
50
64
6
22.369418519393044
24
6000
20000
175
60.0
3000
300
23
300000
9
5.547001962252291
19
16.0
3
2500
5.0
18
240
1350
6.0
6
11990
1000
216.0
750
110
24
1.25
100
95
The coach can divide the team into two groups of 10 players each.
2000
5.0
6
18
37
2000.0
6.666666666666667
3.0
720
22
195
370
5
100
200
144
150
5.5
25.0
30
400
4.0
349
40
8.0
5
10.0
500
625
80
('Pikachu', 'Bulbasaur', 'Charmander')
15.0
47
600.0
2.0
70
2500.0
2
0
1600
480
13
2.0
110
1250.0
33.333333333333336
4.0
21
-1
23.323807579381203
10
40
1.15
24.0
400
21
160
0
24
5
12.5
110
53.333333333333336
4.0
40
18
20.0
10000
4
0.016666666666666666
2.6666666666666665
90
100
4000
7.5
120
240
2
30
96.0
5
36
1000
1
60
100
1.8181818181818181
1000
10.0
37
10
3.0
12
10.0
40.0
8
16777216
1.4
3
5
20
800
20
120
50
51
48.0
5.596
25.0
400
6
20
24
150.0
2.3333333333333335
2
40
37
24
32
10.5
7
22
150
600000
300
1600
120
6

'Percentage of Questions with Executable Code: 7.068129624904312'

In [18]:
result2 = check_topic(df4)
result2

'Percentage of Executable Code that Contains Topic: 88.8086642599278'

In [19]:
print(f"Percentage of questions that have 0 as answer: {(len(df4[df4['answer']==0])/len(df4))*100}")

Percentage of questions that have 0 as answer: 3.6101083032490973


In [20]:
df4 = df4.drop_duplicates(subset='question')
question_numbers = []
for i in range(0, len(df4)):
    question_numbers.append(f"mammoth_q{i}")
df4['question_number'] = question_numbers
#df4_an = df4.sample(n=250, random_state = 42)

In [21]:
df4

,question,solution,answer,topic,question_number
0,Iron Man has 23 suits of armor. He builds 12 m...,def solution():\n #Iron Man has 23 suits of...,31,Iron Man,mammoth_q0
1,Naruto has 100 ninja tools. 20 of them are shu...,def solution():\n #Naruto has 100 ninja too...,50,Naruto,mammoth_q1
2,"Thor, the God of Thunder, can throw his hammer...",def solution():\n #Thor can throw his hamme...,166.666667,"Thor, the God of Thunder",mammoth_q2
3,Iron Man can fly 100 miles per minute. \nHow m...,def solution():\n #Iron Man can fly 100 mil...,500,Iron Man,mammoth_q3
4,Patrick Mahomes is the starting quarterback fo...,def solution():\n #Number of touchdowns thr...,92,Patrick Mahomes,mammoth_q4
...,...,...,...,...,...
272,A child wants to buy ice cream from an ice cre...,def solution():\n # The child has $3.00\n ...,0.5,ice cream,mammoth_q272
273,Dog A barks 12 times a day.\nDog B barks 21 ti...,def solution():\n #Dog A barks 12 times a d...,174,dogs,mammoth_q273
274,The Teenage Mutant Ninja Turtles are having a ...,def solution():\n #Total number of pizza sl...,50,Ninja Turtles,mammoth_q274
275,A bag of candy has 12 pieces. 3/4 of the candy...,def solution():\n #A bag of candy has 12 pi...,3.0,candy,mammoth_q275


# Concat DFs

df = pd.concat([df1, df2, df3, df4])
df = df.sample(frac = 1, random_state = 42)
df_an = pd.concat([df1_an, df2_an, df3_an, df4_an])
df_an = df_an.sample(frac=1, random_state = 42)
#df.to_csv("data\model_questions.csv")
#df_an.to_csv("data\zooniverse_questions.csv")

df

df_an

# Send Each Question to .txt file for Zooniverse Upload

for i in range(0, len(df_an)):
    question_number = df_an.iloc[i]['question_number']
    question = df_an.iloc[i]['question']
    solution = df_an.iloc[i]['solution']
    answer = str(df_an.iloc[i]['answer'])
    topic = df_an.iloc[i]['topic']
    file_name = f"{question_number}.txt"
    file_path = "Zooniverse/final_data/"+file_name
    text = "Topic: "+ topic + "\n\nQuestion:\n" + question + "\n\nSolution:\n" + solution + "\n\nAnswer: " + answer
    with open(file_path, "w") as f:  # Open the file in append mode ("a")
        f.write(text)  # Append the newly generated text to the file